In [1]:
import torch
from torch.autograd import Variable
import torch.utils.data as Data
import torch.nn._functions as F
import matplotlib.pyplot as plt
import torchvision                        # 数据库模块

In [2]:
EPOCH=100
BATCH_SIZE=50
LR=0.001
DOWNLOAD_MNIST=True

train_dataset=torchvision.datasets.MNIST(
    root='mnist',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=DOWNLOAD_MNIST
)

test_dataset=torchvision.datasets.MNIST(
    root='mnist',
    train=False
)

train_loader=Data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

# 为了节约时间, 我们测试时只测试前2000个
test_x = Variable(torch.unsqueeze(test_dataset.test_data, dim=1), volatile=True).type(torch.FloatTensor)[:2000]/255.   # shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_y = test_dataset.test_labels[:2000]
print(test_y.dim())

1


d:\python-venvs\deep_learning\lib\site-packages\ipykernel_launcher.py:26: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


In [3]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 =torch.nn.Sequential(  # input shape (1, 28, 28)
            torch.nn.Conv2d(
                in_channels=1,      # input height
                out_channels=16,    # n_filters
                kernel_size=5,      # filter size
                stride=1,           # filter movement/step
                padding=2,      # 如果想要 con2d 出来的图片长宽没有变化, padding=(kernel_size-1)/2 当 stride=1
            ),                  # output shape (16, 28, 28)
            torch.nn.ReLU(),    # activation
            torch.nn.MaxPool2d(kernel_size=2),    # 在 2x2 空间里向下采样, output shape (16, 14, 14)
        )
        self.conv2 = torch.nn.Sequential(  # input shape (1, 28, 28)
            torch.nn.Conv2d(16, 32, 5, 1, 2),  # output shape (32, 14, 14)
            torch.nn.ReLU(),  # activation
            torch.nn.MaxPool2d(2),  # output shape (32, 7, 7)
        )
        self.out = torch.nn.Linear(32 * 7 * 7, 10)   # fully connected layer, output 10 classes
 
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)   # 展平多维的卷积图成 (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output
 
cnn = CNN()
print(cnn)  # net architecture
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = torch.nn.CrossEntropyLoss()   # the target label is not one-hotted
 
# training and testing
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):   # 分配 batch data, normalize x when iterate train_loader
        b_x = Variable(x)   # batch x
        b_y = Variable(y)   # batch y
 
        output = cnn(b_x)               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
    print('Epoch:',(epoch+1),'| 已完成：%.2f'  % ((epoch+1)*1.0/EPOCH*100),'%')

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)
Epoch: 1 | 已完成：1.00 %
Epoch: 2 | 已完成：2.00 %
Epoch: 3 | 已完成：3.00 %
Epoch: 4 | 已完成：4.00 %
Epoch: 5 | 已完成：5.00 %
Epoch: 6 | 已完成：6.00 %
Epoch: 7 | 已完成：7.00 %
Epoch: 8 | 已完成：8.00 %
Epoch: 9 | 已完成：9.00 %
Epoch: 10 | 已完成：10.00 %
Epoch: 11 | 已完成：11.00 %
Epoch: 12 | 已完成：12.00 %
Epoch: 13 | 已完成：13.00 %
Epoch: 14 | 已完成：14.00 %
Epoch: 15 | 已完成：15.00 %
Epoch: 16 | 已完成：16.00 %
Epoch: 17 | 已完成：17.00 %
Epoch: 18 | 已完成：18.00 %
Epoch: 19 | 已完成：19.00 %
Epoch: 20 | 已完成：20.00 %
Epoch: 21 | 已完成：21.00 %
Epoch: 22 | 已完成：22.00 %
Epoch: 23 | 已完成：23

In [6]:
test_output = cnn(test_x[:10])
print(test_output)
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
print(pred_y, 'prediction number')
print(test_y[:10].numpy(), 'real number')

tensor([[ -82.2359,  -63.0552,  -40.2720,  -14.6442,  -60.5925,  -76.9567,
         -144.1732,   70.3700,  -77.0450,  -29.1047],
        [ -70.6955,  -27.5589,   62.9747,  -91.4573,  -85.5348, -139.8804,
          -18.7168,  -75.3354,  -63.0867,  -88.2515],
        [ -79.3353,   38.8713,  -18.3144,  -65.6569,  -25.3887,  -40.3040,
          -31.7771,  -21.8401,  -17.0369,  -75.8212],
        [  35.5896, -107.5747,  -48.1706,  -60.4682,  -62.4506,  -48.9014,
           -0.4318,  -42.3425,  -27.9472,  -20.7534],
        [ -77.4539,  -75.9994,  -99.5223, -112.7438,   58.7399,  -67.2441,
          -47.7539,  -59.7785,  -41.5259,  -26.4065],
        [ -79.7663,   50.6854,  -21.7826,  -71.2134,  -20.0979,  -54.8645,
          -32.5541,   -8.9323,  -15.3875,  -76.6264],
        [-171.0643,  -51.9632, -101.1390, -124.5959,   60.7161,  -45.3334,
         -135.2560,  -29.9022,  -28.3945,  -35.6035],
        [-110.9425,  -56.9318,  -49.3691,  -85.5600,  -20.5175,  -34.4894,
          -85.0774,  -